In [1]:
!git clone https://github.com/shuntaro0628/Optimization_Project.git
%cd /content/Optimization_Project

Cloning into 'Optimization_Project'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 18 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (18/18), done.
/content/Optimization_Project


In [2]:
# Import libraries
import torch
from torch import tensor
from torch import nn 
from torch import sigmoid
from torch import atan
from torch import tanh
import torch.optim as optim
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import random
import pandas as pd

seed = 7777
random.seed(seed) 
torch.manual_seed(seed)

In [3]:
lpx_min = 50 #mm
lpx_max = 650 #mm
lpy_min = 50 #mm
lpy_max = 2050 #mm
wp_min = 25 # mm
wp_max = 325 # mm
a_min = 0 # mm
a_max = 200 # mm
p_min = 0 # mm
p_max = 200 # mm
ls_min = 50 # mm
ls_max = 450 # mm
ws_min = 25 # mm
ws_max = 225 # mm
Ip_min = 50 # A rms
Ip_max = 200 # A rms
Np_min = 4 #turn
Np_max = 10 #turn
Ns_min = 4 #turn
Ns_max = 10 #turn

In [4]:
class Model(nn.Module): # Design your model using class
    def __init__(self):    
        super(Model, self).__init__() #In the constructor, we instantiate nn.Linear module.
        self.linear1 = nn.Linear(12, 100, bias=True).cuda() # nn.Linear(<input size> ,<output size>)
        self.linear2 = nn.Linear(100, 100, bias=True).cuda()
        self.linear3 = nn.Linear(100, 42, bias=True).cuda()

    def forward(self, x):
        x = atan(self.linear1(x))
        x = atan(self.linear2(x))
        x = atan(self.linear2(x))
        y_pred = self.linear3(x)
        return y_pred

In [5]:
model = Model() # our model
model.load_state_dict(torch.load('saved_model_state.pt'))

for param in model.parameters():
    param.requires_grad = False

In [6]:
def generate_noise(a, b):
    low,high = -1, 1  # range of uniform distribution

    return torch.distributions.uniform.Uniform(low,high).sample([a,b]).cuda()

In [7]:
from torch.nn.modules import activation

class Generator(nn.Module):

    def __init__(self, input_length: int): 
      super(Generator, self).__init__() 
      self.dense_layer = nn.Linear(int(input_length), 128).cuda() 
      torch.nn.init.kaiming_uniform_(self.dense_layer.weight) 
      self.bn1 = nn.BatchNorm1d(128).cuda()
      self.dense_layer1 = nn.Linear(128, 256).cuda() 
      torch.nn.init.kaiming_uniform_(self.dense_layer1.weight) 
      self.bn2 = nn.BatchNorm1d(256).cuda()
      self.dense_layer2 = nn.Linear(256, 512).cuda() 
      torch.nn.init.kaiming_uniform_(self.dense_layer2.weight) 
      self.bn3 = nn.BatchNorm1d(512).cuda()
      self.dense_layer3 = nn.Linear(512, 256).cuda() 
      torch.nn.init.kaiming_uniform_(self.dense_layer3.weight) 
      self.bn4 = nn.BatchNorm1d(256).cuda()
      self.dense_layer4 = nn.Linear(256, 128).cuda() 
      torch.nn.init.kaiming_uniform_(self.dense_layer4.weight) 
      self.bn5 = nn.BatchNorm1d(128).cuda()
      self.dense_layer5 = nn.Linear(128, int(input_length)).cuda() 
      torch.nn.init.kaiming_uniform_(self.dense_layer5.weight) 
      self.activation = nn.Sigmoid() 

    def forward(self, x): 
      x = atan(self.bn1(self.dense_layer(x)))
      x = atan(self.bn2(self.dense_layer1(x)))
      x = atan(self.bn3(self.dense_layer2(x)))
      x = atan(self.bn4(self.dense_layer3(x))) 
      x = atan(self.bn5(self.dense_layer4(x))) 
      x = self.activation(self.dense_layer5(x)) 
      return x 

In [8]:
generator = Generator(10)
for param in generator.parameters():
    param.requires_grad = True

In [9]:
def scaler(value, min , max):
    return value * (max - min) + min

In [10]:
@torch.jit.script
def linspace(start: torch.Tensor, stop: torch.Tensor, num: int):
    """
    Creates a tensor of shape [num, *start.shape] whose values are evenly spaced from start to end, inclusive.
    Replicates but the multi-dimensional bahaviour of numpy.linspace in PyTorch.
    """
    # create a tensor of 'num' steps from 0 to 1
    steps = torch.arange(num, dtype=torch.float32, device=start.device) / (num - 1)
    
    # reshape the 'steps' tensor to [-1, *([1]*start.ndim)] to allow for broadcastings
    # - using 'steps.reshape([-1, *([1]*start.ndim)])' would be nice here but torchscript
    #   "cannot statically infer the expected size of a list in this contex", hence the code below
    for i in range(start.ndim):
        steps = steps.unsqueeze(-1)
    
    # the output starts at 'start' and increments until 'stop' in each dimension
    out = start[None] + steps*(stop - start)[None]
    
    return out

In [11]:
def scale_and_extract(arr):
    # scaler_min = torch.tensor([50., 50., 25., 0., 0., 50., 25.,50.,4.,4.], requires_grad=True, device="cuda")
    # scaler_max = torch.tensor([2050., 650., 325., 200., 200., 450., 225.,200.,10.,10.], requires_grad=True, device="cuda")

    scaler_min = torch.tensor([0., 50., 50., 0., 50., 0., 25.,50.,4.,4.], requires_grad=True, device="cuda")
    scaler_max = torch.tensor([200., 650., 2050., 450., 200., 325., 225.,200.,10.,10.], requires_grad=True, device="cuda")
    
    arr = scaler(arr, scaler_min, scaler_max)
    y0_i = (arr[:,2] + arr[:,5]) / 2
    y1_i = (3/2 * arr[:,2]) + (5/2 * arr[:,5]) + (2 * arr[:,0]) + (arr[:,4])

    ys_min = (y1_i)
    ys_max = (y1_i + (y1_i - y0_i) / 2)
    
    ys = linspace(ys_min, ys_max, num = 5)
    ys = torch.transpose(ys, 0, 1)
    arr[:,8:10] = torch.round(arr[:,8:10])
    temp = torch.cat([arr, ys], axis=1)
    return temp


In [12]:
from torch.optim.lr_scheduler import ExponentialLR
TRAINING = 3000
learning_rate = 1e-3
optimizer = torch.optim.Adam(generator.parameters(), lr=learning_rate)

#optimizer = torch.optim.Adam(generator.parameters(), lr = learning_rate)

In [13]:
kdiffs = []
coil_losses = []

In [14]:
import math
f = 85*10**3 #[Hz]
w =2*math.pi*f #[rad/s]
Pout=50000 #W
Vdc = 400 #800 # V
Vbat = 400 #V
QCoil = 400
b = 50

In [15]:
noise = generate_noise(10000, 10)
GP = generator(noise)
scaled_array = scale_and_extract(GP)
gp_parameters = torch.cat([scaled_array[:,0:7],scaled_array[:,10:15]],axis=1)
extra_parameters = scaled_array[:,7:10]
print(extra_parameters)
MP = model(scaled_array[:,0:12])


tensor([[175.6591,   6.0000,   7.0000],
        [ 80.7429,   9.0000,   9.0000],
        [ 57.5141,   9.0000,   7.0000],
        ...,
        [ 91.8971,   8.0000,   9.0000],
        [ 82.6505,   9.0000,   8.0000],
        [162.1917,   5.0000,   5.0000]], device='cuda:0',
       grad_fn=<SliceBackward0>)


In [16]:
def min_max(x_sample, x, axis=0):# axis=0 is vertical direction.
    min_x = x_sample.min(axis=axis, keepdims=True)
    max_x = x_sample.max(axis=axis, keepdims=True)
    x = (x-min_x.values)/(max_x.values-min_x.values)
    print(x)

def inv_min_max(x_sample, x_std, axis=0):
    min_x = x_sample.min(axis=axis, keepdims=True)
    max_x = x_sample.max(axis=axis, keepdims=True)
    x = x_std*(max_x.values-min_x.values)+min_x.values


In [17]:
foldername ='/content/Optimization_Project/' 
filename1 = 'DWPT_v5_N10'
filename2 = 'DWPT_v5_N100'
filename3 = 'DWPT_v5_N200'
filename4 = 'DWPT_v5_N300'
filename5 = 'DWPT_v5_N400'
df1 = pd.read_csv(foldername+filename1+'_after.csv')
df2 = pd.read_csv(foldername+filename2+'_after.csv')
df3 = pd.read_csv(foldername+filename3+'_after.csv')
df4 = pd.read_csv(foldername+filename4+'_after.csv')
df5 = pd.read_csv(foldername+filename5+'_after.csv')
df = pd.concat([df1,df2,df3,df4,df5],axis=0)

In [18]:
del df['Unnamed: 0']

In [ ]:

torch.autograd.set_detect_anomaly(True)

for i in range(TRAINING):

    for param in generator.parameters():
        param.grad = None

    noise = generate_noise(10000, 10) #IP, Ns and Np
    GP = generator(noise)

    scaled_array = scale_and_extract(GP)

    gp_parameters = torch.cat([scaled_array[:,0:7],scaled_array[:,10:15]],axis=1)

    x = df.values[:,0:12]
    #temp_gp = gp_parameters.clone()
    x = torch.tensor(x, dtype = torch.float, requires_grad=True).cuda()

    min_x = torch.min(x, dim = 0, keepdim=True).values
    max_x = torch.max(x, dim = 0, keepdim=True).values

    #value * (max - min) + min
    gp_parameters = (gp_parameters.clone()-min_x)/(max_x-min_x)
    # print(gp_parameters)
    #gp_parameters = gp_parameters.clone()*(max_x-min_x) + min_x
    extra_parameters = scaled_array[:, 7:10] #Ip, Np, Ns

    # print(gp_parameters[0])
    # print(min_x)
    # print(max_x)
    # break
    MP = model(gp_parameters)
    # y_df = df.loc[:, 'k0mm_ys0':]
    # print(y_df.columns)

    # y = torch.tensor(y_df.values, dtype=torch.float).cuda()

    # min_x = y.min(axis=0, keepdims=True)
    # max_x = y.max(axis=0, keepdims=True)
    # MP = MP*(max_x.values-min_x.values)+min_x.values
    # print(MP)
    y_df = df.loc[:, 'k0mm_ys0':]

    y = y_df.values
    y = torch.tensor(y, dtype = torch.float, requires_grad=True).cuda()

    min_y = torch.min(y, dim = 0).values
    max_y = torch.max(y, dim = 0).values

    MP = (MP.clone()*(max_y-min_y)).clone() + min_y
    #print(MP[0], GP[0])
    #calculate Is 
    kdiff = MP[:, 0].clone() - MP[:, 5].clone()
    kdiff = abs(kdiff / MP[:, 0].clone())
    kdiff = torch.mean(kdiff) # + scaling factor * Bstray Loss 
    kdiffs.append(kdiff.detach().cpu().item())
    #kdiff.backward()
    #print(MP[:,11]) # LS0MM_YSO
    #First consider only Bstray 
    # optimizer.step()
    #print(MP[:,6]) #LP0MM_YS0

    #1/(lpy_i+2*wp_i+2*a_i+p_i)
    # num_inverters = 1/(temp_gp[:,2]+2*temp_gp[:,5]+2*temp_gp[:,0]+temp_gp[:,4])*10**3

    # sum_ = torch.mean(num_inverters)
    # min_x = torch.min(torch.tensor(y_df.values, dtype=torch.float), dim=0).values.cuda()
    # max_x = torch.max(torch.tensor(y_df.values, dtype=torch.float), dim=0).values.cuda()


    MP[:,6] = MP[:,6].clone() * extra_parameters[:,1].clone()**2  # LP0MM_YSO
    MP[:,7] = MP[:,7].clone() * extra_parameters[:,1].clone()**2  # LP0MM_YS1
    MP[:,8] = MP[:,8].clone() * extra_parameters[:,1].clone()**2  # LP0MM_YS2
    MP[:,9] = MP[:,9].clone() * extra_parameters[:,1].clone()**2  # LP0MM_YS3
    MP[:,10] = MP[:,10].clone() * extra_parameters[:,1].clone()**2 # LP0MM_YS4

    MP[:,11] = MP[:,11].clone() * extra_parameters[:,2].clone()**2 # LS0MM_YSO
    MP[:,12] = MP[:,12].clone() * extra_parameters[:,2].clone()**2 # LS0MM_YS1
    MP[:,13] = MP[:,13].clone() * extra_parameters[:,2].clone()**2 # LS0MM_YS2
    MP[:,14] = MP[:,14].clone() * extra_parameters[:,2].clone()**2 # LS0MM_YS3
    MP[:,15] = MP[:,15].clone() * extra_parameters[:,2].clone()**2 # LS0MM_YS4

    n1_i = math.pi*w*MP[:,6]*10**(-9)*extra_parameters[:,0]*math.sqrt(2)/(4*Vdc)
    t1 = (math.pi)**2*w*(MP[:,6]*10**(-9)*MP[:,11] *10**(-9))**0.5
    t2 = Pout/(8*MP[:,0]*n1_i*Vdc*Vbat)
    #print(t1, t2)
    n2_i = t1*t2
    #print(n2_i)
    Is_i = 4*n2_i*Vbat/(math.pi*w*MP[:,11]*10**(-9))/math.sqrt(2)

    coil_loss = w*MP[:,6]*10**(-9)*extra_parameters[:,0]**2/QCoil + w*MP[:,11] *10**(-9)*Is_i**2/QCoil

    bx = (((MP[:, 30]*extra_parameters[:, 0]*extra_parameters[:, 1]) + (MP[:, 36]*Is_i*extra_parameters[:, 2]))**2 + ((MP[:, 33]*extra_parameters[:, 0]*extra_parameters[:, 1]) + (MP[:, 39]*Is_i*extra_parameters[:, 2]))**2)**0.5

    by = (((MP[:, 31]*extra_parameters[:, 0]*extra_parameters[:, 1]) + (MP[:, 37]*Is_i*extra_parameters[:, 2]))**2 + ((MP[:, 34]*extra_parameters[:, 0]*extra_parameters[:, 1]) + (MP[:, 40]*Is_i*extra_parameters[:, 2]))**2)**0.5

    bz = (((MP[:, 32]*extra_parameters[:, 0]*extra_parameters[:, 1])+ (MP[:, 38]*Is_i*extra_parameters[:, 2]))**2 + ((MP[:, 35]*extra_parameters[:, 0]*extra_parameters[:, 1]) + (MP[:, 41]*Is_i*extra_parameters[:, 2]))**2)**0.5

    bstray = ((bx**2 + by**2 + bz**2)**0.5)

    p0 = w*MP[:,0]*(MP[:,6]*10**(-9))*(MP[:,11]*10**(-9))**0.5*extra_parameters[:,0]*Is_i
    p1 = w*MP[:,1]*(MP[:,7]*10**(-9))*(MP[:,12]*10**(-9))**0.5*extra_parameters[:,0]*Is_i  
    p2 = w*MP[:,2]*(MP[:,8]*10**(-9))*(MP[:,13]*10**(-9))**0.5*extra_parameters[:,0]*Is_i 
    p3 = w*MP[:,3]*(MP[:,9]*10**(-9))*(MP[:,14]*10**(-9))**0.5*extra_parameters[:,0]*Is_i
    p4 = w*MP[:,4]*(MP[:,10]*10**(-9))*(MP[:,15]*10**(-9))**0.5*extra_parameters[:,0]*Is_i 
    pave = (p0+(2*p1)+(2*p2)+(2*p3)+(2*abs(p4)))/8 

    V_PriCore = (gp_parameters[:,2] +2*gp_parameters[:,5]+2*gp_parameters[:,0])*(gp_parameters[:,1]+2*gp_parameters[:,5]+2*gp_parameters[:,0])*5/(10**3) # cm3
    V_SecCore = (gp_parameters[:,3]+2*gp_parameters[:,6]+2*b)**2*5/(10**3) # cm3

    V_PriWind = (2*(gp_parameters[:,1]+gp_parameters[:,5])+2*(gp_parameters[:,2]+gp_parameters[:,5]))*6.6*6.6/(10**3)*extra_parameters[:,1]
    V_SecWind = 4*(gp_parameters[:,3]+gp_parameters[:,6])*6.6*6.6/(10**3)*extra_parameters[:,2]

    V_PriWind_ave = V_PriWind/(gp_parameters[:,2]+2*gp_parameters[:,5]+2*gp_parameters[:,0]+gp_parameters[:,4])*10**3 # cm3/m
    V_PriCore_ave = V_PriCore/(gp_parameters[:,2]+2*gp_parameters[:,5]+2*gp_parameters[:,0]+gp_parameters[:,4])*10**3 # cm3/m

    mean_V_PriCore_ave = torch.mean(V_PriCore_ave)
    mean_V_SecCore = torch.mean(V_SecCore)
    mean_V_PriWind_ave = torch.mean(V_PriWind_ave)
    mean_V_SecWind = torch.mean(V_SecWind)

    mean_coil_loss = torch.mean(coil_loss)
    mean_bstray = torch.mean(bstray)
    mean_pave = torch.mean(pave)
    coil_losses.append(mean_coil_loss.detach().cpu().item())
    # combined_loss = mean_coil_loss + kdiff + mean_bstray - mean_pave - mean_V_PriWind - mean_V_SecWind - mean_V_PriCore - mean_V_SecCore
    combined_loss = mean_coil_loss + kdiff + mean_bstray - mean_pave
    combined_loss.backward()
    optimizer.step()

    if i % 100 == 0:
        print(f"Epoch: {i:4d} | Coupling: {100 * kdiff.item():.3f}%")
        print(f"Epoch: {i:4d} | Coil Loss: {mean_coil_loss.item():.3f}")
        print(f"Epoch: {i:4d} | Bstray: {mean_bstray.item():.3f}")
        print(f"Epoch: {i:4d} | Pave: {mean_pave.item():.3f}")
        print(f"Epoch: {i:4d} | V_PriCore: {mean_V_PriCore_ave.item():.3f}")
        print(f"Epoch: {i:4d} | V_SecCore: {mean_V_SecCore.item():.3f}")
        print(f"Epoch: {i:4d} | V_PriWind: {mean_V_PriWind_ave.item():.3f}")
        print(f"Epoch: {i:4d} | V_SecWind: {mean_V_SecWind.item():.3f}")
        #print(f"Epoch: {i:4d} | Inverter: {sum_.item():.3f}")
        
    # scaler_min = torch.tensor([0., 50., 50., 0., 50., 0., 25.,50.,4.,4.], requires_grad=True, device="cuda")
    # scaler_max = torch.tensor([200., 650., 2050., 450., 200., 325., 225.,200.,10.,10.], requires_grad=True, device="cuda")

Epoch:    0 | Coupling: 13.138%
Epoch:    0 | Coil Loss: 2879.691
Epoch:    0 | Bstray: 138.118
Epoch:    0 | Pave: 361.350
Epoch:    0 | V_PriCore: 9.411
Epoch:    0 | V_SecCore: 51.435
Epoch:    0 | V_PriWind: 386.671
Epoch:    0 | V_SecWind: 1.131
Epoch:  100 | Coupling: 13.042%
Epoch:  100 | Coil Loss: 1255.191
Epoch:  100 | Bstray: 52.212
Epoch:  100 | Pave: 388.276
Epoch:  100 | V_PriCore: 9.967
Epoch:  100 | V_SecCore: 51.823
Epoch:  100 | V_PriWind: 411.098
Epoch:  100 | V_SecWind: 1.456
Epoch:  200 | Coupling: 10.110%
Epoch:  200 | Coil Loss: 878.772
Epoch:  200 | Bstray: 61.392
Epoch:  200 | Pave: 312.284
Epoch:  200 | V_PriCore: 10.280
Epoch:  200 | V_SecCore: 52.466
Epoch:  200 | V_PriWind: 500.314
Epoch:  200 | V_SecWind: 1.917
Epoch:  300 | Coupling: 7.783%
Epoch:  300 | Coil Loss: 595.804
Epoch:  300 | Bstray: 60.729
Epoch:  300 | Pave: 236.628
Epoch:  300 | V_PriCore: 9.873
Epoch:  300 | V_SecCore: 52.746
Epoch:  300 | V_PriWind: 632.750
Epoch:  300 | V_SecWind: 2.156
E

In [ ]:

p0 = w*MP[:,0]*(MP[:,6]*10**(-9)*MP[:,11]*10**(-9))**0.5*extra_parameters[:,0]*Is_i
p1 = w*MP[:,1]*(MP[:,7]*10**(-9)*MP[:,12]*10**(-9))**0.5*extra_parameters[:,0]*Is_i  
p2 = w*MP[:,2]*(MP[:,8]*10**(-9)*MP[:,13]*10**(-9))**0.5*extra_parameters[:,0]*Is_i 
p3 = w*MP[:,3]*(MP[:,9]*10**(-9)*MP[:,14]*10**(-9))**0.5*extra_parameters[:,0]*Is_i 
p4 = w*MP[:,4]*(MP[:,10]*10**(-9)*MP[:,15]*10**(-9))**0.5*extra_parameters[:,0]*Is_i 
pave = (p0+2*p1+2*p2+2*p3+2*abs(p4))/8 

b = 50

V_PriCore = (gp_parameters[:,2] +2*gp_parameters[:,5]+2*gp_parameters[:,0])*(gp_parameters[:,1]+2*gp_parameters[:,5]+2*gp_parameters[:,0])*5/(10**3) # cm3
V_SecCore = (gp_parameters[:,3]+2*gp_parameters[:,6]+2*b)**2*5/(10**3) # cm3

V_PriWind = (2*(gp_parameters[:,1]+gp_parameters[:,5])+2*(gp_parameters[:,2]+gp_parameters[:,5]))*6.6*6.6/(10**3)*extra_parameters[:,1]
V_SecWind = 4*(gp_parameters[:,3]+gp_parameters[:,6])*6.6*6.6/(10**3)*extra_parameters[:,2]

V_PriWind_ave = V_PriWind/(gp_parameters[:,2]+2*gp_parameters[:,5]+2*gp_parameters[:,0]+gp_parameters[:,4])*10**3 # cm3/m

V_PriCore_ave = V_PriCore/(gp_parameters[:,2]+2*gp_parameters[:,5]+2*gp_parameters[:,0]+gp_parameters[:,4])*10**3 # cm3/m


In [ ]:
with torch.no_grad():
  noise = generate_noise(10000, 10) #IP, Ns and Np
  GP = generator(noise)

  scaled_array = scale_and_extract(GP)

  gp_parameters = torch.cat([scaled_array[:,0:7],scaled_array[:,10:15]],axis=1)
  temp_gp = gp_parameters.clone()
  x = df.values[:,0:12]

  x = torch.tensor(x, dtype = torch.float, requires_grad=True).cuda()

  min_x = torch.min(x, dim = 0, keepdim=True).values
  max_x = torch.max(x, dim = 0, keepdim=True).values

  #value * (max - min) + min
  gp_parameters = (gp_parameters.clone()-min_x)/(max_x-min_x)
  print(gp_parameters)
  #gp_parameters = gp_parameters.clone()*(max_x-min_x) + min_x
  extra_parameters = scaled_array[:, 7:10] #Ip, Np, Ns
  MP = model(gp_parameters)
  # y_df = df.loc[:, 'k0mm_ys0':]
  # print(y_df.columns)

  # y = torch.tensor(y_df.values, dtype=torch.float).cuda()

  # min_x = y.min(axis=0, keepdims=True)
  # max_x = y.max(axis=0, keepdims=True)
  # MP = MP*(max_x.values-min_x.values)+min_x.values
  # print(MP)
  y_df = df.loc[:, 'k0mm_ys0':]

  y = y_df.values
  y = torch.tensor(y, dtype = torch.float, requires_grad=True).cuda()

  min_y = torch.min(y, dim = 0).values
  max_y = torch.max(y, dim = 0).values

  MP = (MP.clone()*(max_y-min_y)).clone() + min_y

  print(MP[0])
  # print(noise[0], noise[9000])
  # print(GP[0], GP[9000])
  # print(MP[1], MP[9000])

In [ ]:
df_x_final=pd.DataFrame(temp_gp.detach().cpu().numpy(), columns=['a[mm]', 'lpx[mm]', 'lpy[mm]', 'ls[mm]',\
  'p[mm]', 'wp[mm]', 'ws[mm]', 'ys0[mm]', 'ys1[mm]', 'ys2[mm]', 'ys3[mm]', 'ys4[mm]'])

cls=['k0mm_ys0','k0mm_ys1', 'k0mm_ys2', 'k0mm_ys3', 'k0mm_ys4', 'k100mm_ys0',\
 'Lp0mm_ys0[nH]', 'Lp0mm_ys1[nH]', 'Lp0mm_ys2[nH]', 'Lp0mm_ys3[nH]', 'Lp0mm_ys4[nH]',\
 'Ls0mm_ys0[nH]', 'Ls0mm_ys1[nH]', 'Ls0mm_ys2[nH]', 'Ls0mm_ys3[nH]',\
 'Ls0mm_ys4[nH]', 'Lp100mm_ys0[nH]', 'Ls100mm_ys0[nH]', 'Bx_p0mm0deg[uT]',\
 'By_p0mm0deg[uT]', 'Bz_p0mm0deg[uT]', 'Bx_p0mm90deg[uT]', 'By_p0mm90deg[uT]',\
 'Bz_p0mm90deg[uT]', 'Bx_s0mm0deg[uT]', 'By_s0mm0deg[uT]', 'Bz_s0mm0deg[uT]',\
 'Bx_s0mm90deg[uT]', 'By_s0mm90deg[uT]', 'Bz_s0mm90deg[uT]',\
 'Bx_p100mm0deg[uT]', 'By_p100mm0deg[uT]', 'Bz_p100mm0deg[uT]',\
 'Bx_p100mm90deg[uT]', 'By_p100mm90deg[uT]', 'Bz_p100mm90deg[uT]',\
 'Bx_s100mm0deg[uT]', 'By_s100mm0deg[uT]', 'Bz_s100mm0deg[uT]',\
 'Bx_s100mm90deg[uT]', 'By_s100mm90deg[uT]', 'Bz_s100mm90deg[uT]']
df_y_pred=pd.DataFrame(MP.detach().cpu().numpy(), columns=cls)



In [ ]:
df_I_N = pd.DataFrame(extra_parameters[:,0].detach().cpu().numpy(), columns = ['Ip[A(rms)]'])

In [ ]:
df_turn_N = pd.DataFrame(extra_parameters[:,1:].detach().cpu().numpy(), columns = ['Np[turn]','Ns[turn]'])

In [ ]:
df_pred = pd.concat([df_I_N, df_turn_N, df_x_final, df_y_pred],axis=1)

In [ ]:
df_pred

In [ ]:
#Calculate B field from prediction
N = 10000
df_pred = pd.concat([df_I_N, df_turn_N, df_x_final, df_y_pred],axis=1)
Bx_p0mm0deg =[]
By_p0mm0deg =[]
Bz_p0mm0deg =[]

Bx_p0mm90deg =[]
By_p0mm90deg =[]
Bz_p0mm90deg =[]

Bx_p100mm0deg =[]
By_p100mm0deg =[]
Bz_p100mm0deg =[]

Bx_p100mm90deg =[]
By_p100mm90deg =[]
Bz_p100mm90deg =[]

Bx_s0mm0deg =[]
By_s0mm0deg =[]
Bz_s0mm0deg =[]

Bx_s0mm90deg =[]
By_s0mm90deg =[]
Bz_s0mm90deg =[]

Bx_s100mm0deg =[]
By_s100mm0deg =[]
Bz_s100mm0deg =[]

Bx_s100mm90deg =[]
By_s100mm90deg =[]
Bz_s100mm90deg =[]

import math
f = 85*10**3 #[Hz]
w =2*math.pi*f #[rad/s]
Pout=50000 #W
Vdc = 400 #800 # V
Vbat = 400 #V
Ip =[]
Is =[]
VLp = []
VLs = []
Np =[]
Ns =[]
Lp0mm_ys0=[]
Lp0mm_ys1=[]
Lp0mm_ys2=[]
Lp0mm_ys3=[]
Lp0mm_ys4=[]
Ls0mm_ys0=[]
Ls0mm_ys1=[]
Ls0mm_ys2=[]
Ls0mm_ys3=[]
Ls0mm_ys4=[]
Lp100mm_ys0=[]
Ls100mm_ys0=[]
k0mm_ys0=[]
k0mm_ys1=[]
k0mm_ys2=[]
k0mm_ys3=[]
k0mm_ys4=[]
k100mm_ys0=[]
kdiff=[]
Pdiff = []

df_Is = pd.DataFrame(index=[], columns = ['Is[A(rms)]'])
for i in range(len(df_pred.loc[:, 'Bx_p0mm0deg[uT]'])):
  Bx_p0mm0deg.append(df_pred.loc[i, 'Bx_p0mm0deg[uT]']) #0-Peak
  By_p0mm0deg.append(df_pred.loc[i, 'By_p0mm0deg[uT]'])
  Bz_p0mm0deg.append(df_pred.loc[i, 'Bz_p0mm0deg[uT]'])

  Bx_p0mm90deg.append(df_pred.loc[i, 'Bx_p0mm90deg[uT]'])
  By_p0mm90deg.append(df_pred.loc[i, 'By_p0mm90deg[uT]'])
  Bz_p0mm90deg.append(df_pred.loc[i, 'Bz_p0mm90deg[uT]'])

  Bx_p100mm0deg.append(df_pred.loc[i, 'Bx_p100mm0deg[uT]'])
  By_p100mm0deg.append(df_pred.loc[i, 'By_p100mm0deg[uT]'])
  Bz_p100mm0deg.append(df_pred.loc[i, 'Bz_p100mm0deg[uT]'])

  Bx_p100mm90deg.append(df_pred.loc[i, 'Bx_p100mm90deg[uT]'])
  By_p100mm90deg.append(df_pred.loc[i, 'By_p100mm90deg[uT]'])
  Bz_p100mm90deg.append(df_pred.loc[i, 'Bz_p100mm90deg[uT]'])

  Bx_s0mm0deg.append(df_pred.loc[i, 'Bx_s0mm0deg[uT]'])
  By_s0mm0deg.append(df_pred.loc[i, 'By_s0mm0deg[uT]'])
  Bz_s0mm0deg.append(df_pred.loc[i, 'Bz_s0mm0deg[uT]'])

  Bx_s0mm90deg.append(df_pred.loc[i, 'Bx_s0mm90deg[uT]'])
  By_s0mm90deg.append(df_pred.loc[i, 'By_s0mm90deg[uT]'])
  Bz_s0mm90deg.append(df_pred.loc[i, 'Bz_s0mm90deg[uT]'])

  Bx_s100mm0deg.append(df_pred.loc[i, 'Bx_s100mm0deg[uT]'])
  By_s100mm0deg.append(df_pred.loc[i, 'By_s100mm0deg[uT]'])
  Bz_s100mm0deg.append(df_pred.loc[i, 'Bz_s100mm0deg[uT]'])

  Bx_s100mm90deg.append(df_pred.loc[i, 'Bx_s100mm90deg[uT]'])
  By_s100mm90deg.append(df_pred.loc[i, 'By_s100mm90deg[uT]'])
  Bz_s100mm90deg.append(df_pred.loc[i, 'Bz_s100mm90deg[uT]'])

  Np.append(df_turn_N.loc[i, 'Np[turn]'])
  Ns.append(df_turn_N.loc[i, 'Ns[turn]'])  

  Lp0mm_ys0.append(df_pred.loc[i, 'Lp0mm_ys0[nH]']*df_turn_N.loc[i, 'Np[turn]']**2)# The datatype is list.
  Lp0mm_ys1.append(df_pred.loc[i, 'Lp0mm_ys1[nH]']*df_turn_N.loc[i, 'Np[turn]']**2) # nH
  Lp0mm_ys2.append(df_pred.loc[i, 'Lp0mm_ys2[nH]']*df_turn_N.loc[i, 'Np[turn]']**2)
  Lp0mm_ys3.append(df_pred.loc[i, 'Lp0mm_ys3[nH]']*df_turn_N.loc[i, 'Np[turn]']**2)
  Lp0mm_ys4.append(df_pred.loc[i, 'Lp0mm_ys4[nH]']*df_turn_N.loc[i, 'Np[turn]']**2)
  
  Ls0mm_ys0.append(df_pred.loc[i, 'Ls0mm_ys0[nH]']*df_turn_N.loc[i, 'Ns[turn]']**2)
  Ls0mm_ys1.append(df_pred.loc[i, 'Ls0mm_ys1[nH]']*df_turn_N.loc[i, 'Ns[turn]']**2)
  Ls0mm_ys2.append(df_pred.loc[i, 'Ls0mm_ys2[nH]']*df_turn_N.loc[i, 'Ns[turn]']**2)
  Ls0mm_ys3.append(df_pred.loc[i, 'Ls0mm_ys3[nH]']*df_turn_N.loc[i, 'Ns[turn]']**2)
  Ls0mm_ys4.append(df_pred.loc[i, 'Ls0mm_ys4[nH]']*df_turn_N.loc[i, 'Ns[turn]']**2)
  
  Lp100mm_ys0.append(df_pred.loc[i, 'Lp100mm_ys0[nH]']*df_turn_N.loc[i, 'Np[turn]']**2)
  Ls100mm_ys0.append(df_pred.loc[i, 'Ls100mm_ys0[nH]']*df_turn_N.loc[i, 'Ns[turn]']**2) 
  
  k0mm_ys0.append(df_pred.loc[i, 'k0mm_ys0'])
  k0mm_ys1.append(df_pred.loc[i, 'k0mm_ys1'])
  k0mm_ys2.append(df_pred.loc[i, 'k0mm_ys2'])
  k0mm_ys3.append(df_pred.loc[i, 'k0mm_ys3'])
  k0mm_ys4.append(df_pred.loc[i, 'k0mm_ys4'])    
  k100mm_ys0.append(df_pred.loc[i, 'k100mm_ys0'])  

  Ip.append(df_I_N.loc[i,'Ip[A(rms)]']) #A(rms)

  if i % 1000 ==0:
   print(f'Calculating Is... {i}/{N}')

for i in range(len(Ip)):
  n1_i = math.pi*w*Lp0mm_ys0[i]*10**(-9)*Ip[i]*math.sqrt(2)/(4*Vdc) # Note that Ip is rms value. #everything would be numpy
  n2_i = (math.pi)**2*w*(Lp0mm_ys0[i]*10**(-9)*Ls0mm_ys0[i]*10**(-9))**0.5*Pout/(8*k0mm_ys0[i]*n1_i*Vdc*Vbat)
  Is_i = 4*n2_i*Vbat/(math.pi*w*Ls0mm_ys0[i]*10**(-9))/math.sqrt(2)
  VLp_i =( (((4*(Lp0mm_ys0[i])**0.5*Vbat*k0mm_ys0[i]*n2_i)/(math.pi*(Ls0mm_ys0[i])**0.5))**2 + (4*Vdc *n1_i/math.pi)**2)**0.5 )/(2**0.5) # Vrms
  VLs_i =( (((4*(Ls0mm_ys0[i])**0.5*Vdc *k0mm_ys0[i]*n1_i)/(math.pi*(Lp0mm_ys0[i])**0.5))**2 + (4*Vbat*n2_i/math.pi)**2)**0.5 )/(2**0.5) # Vrms
  Is.append(Is_i)
  VLp.append(VLp_i)
  VLs.append(VLs_i)
  df_Is_temp = pd.DataFrame({'Is[A(rms)]':Is_i}, index=[i])
  df_Is = pd.concat([df_Is, df_Is_temp])

df_I = pd.concat([df_I_N, df_Is],axis=1)

Pout_0mm=[]
Pout_100mm=[]
Pave=[]
Pdiff=[]
Pout = []

Bx_0mm=[]
By_0mm=[]
Bz_0mm=[]
Bx_100mm=[]
By_100mm=[]
Bz_100mm=[]
Bstray=[]
CoilLoss_100mm=[]
Qcoil=400

for i in range(len(Bx_p0mm0deg)):#=N  
  Bx_100mm.append(((Bx_p100mm0deg[i]*Ip[i]*Np[i]+ Bx_s100mm0deg[i]*Is[i]*Ns[i] )**2 +(Bx_p100mm90deg[i]*Ip[i]*Np[i]+ Bx_s100mm90deg[i]*Is[i]*Ns[i] )**2)**0.5)
  By_100mm.append(((By_p100mm0deg[i]*Ip[i]*Np[i]+ By_s100mm0deg[i]*Is[i]*Ns[i] )**2 +(By_p100mm90deg[i]*Ip[i]*Np[i]+ By_s100mm90deg[i]*Is[i]*Ns[i] )**2)**0.5)
  Bz_100mm.append(((Bz_p100mm0deg[i]*Ip[i]*Np[i]+ Bz_s100mm0deg[i]*Is[i]*Ns[i] )**2 +(Bz_p100mm90deg[i]*Ip[i]*Np[i]+ Bz_s100mm90deg[i]*Is[i]*Ns[i] )**2)**0.5)
  
  Pout_0mm.append(w*k0mm_ys0[i]*(Lp0mm_ys0[i]*10**(-9)*Ls0mm_ys0[i]*10**(-9))**0.5*Ip[i]*Is[i] ) #[W]  
  Pout_0mm_ys0_i = w*k0mm_ys0[i]*(Lp0mm_ys0[i]*10**(-9)*Ls0mm_ys0[i]*10**(-9))**0.5*Ip[i]*Is[i] #[W] 
  Pout_0mm_ys1_i = w*k0mm_ys1[i]*(Lp0mm_ys1[i]*10**(-9)*Ls0mm_ys1[i]*10**(-9))**0.5*Ip[i]*Is[i] #[W]  
  Pout_0mm_ys2_i = w*k0mm_ys2[i]*(Lp0mm_ys2[i]*10**(-9)*Ls0mm_ys2[i]*10**(-9))**0.5*Ip[i]*Is[i] #[W]  
  Pout_0mm_ys3_i = w*k0mm_ys3[i]*(Lp0mm_ys3[i]*10**(-9)*Ls0mm_ys3[i]*10**(-9))**0.5*Ip[i]*Is[i] #[W]  
  Pout_0mm_ys4_i = w*k0mm_ys4[i]*(Lp0mm_ys4[i]*10**(-9)*Ls0mm_ys4[i]*10**(-9))**0.5*Ip[i]*Is[i] #[W]  
  Pave_i = (Pout_0mm_ys0_i+2*Pout_0mm_ys1_i+2*Pout_0mm_ys2_i+2*Pout_0mm_ys3_i+2*abs(Pout_0mm_ys4_i))/8 # [W]/m
  Pave.append(Pave_i)
  if i % 1000 == 0:
   print(f'Calculating Bstray... {i}/{N}')
B_100mm = [(x**2 + y**2 + z**2)**0.5 for (x,y,z) in zip(Bx_100mm,By_100mm,Bz_100mm)] #[uT(rms)]

df_pred_processed=df_pred
df_pred_processed.insert(0, 'Is[A(rms)]', Is)
df_pred_processed.insert(0, 'VLp[V(rms)]', VLp)
df_pred_processed.insert(0, 'VLs[V(rms)]', VLs)
df_pred_processed.insert(0, 'Bstray[uT(rms)]', B_100mm)
df_pred_processed.insert(0, 'Pout_0mm[W]', Pout_0mm)
df_pred_processed.insert(0, 'Pave[W/m]', Pave)

In [ ]:
kdiff =[abs((x-y)/x)*100 for (x,y) in zip(k0mm_ys0,k100mm_ys0)]
CoilLoss_0mm = [w*x1*10**(-9)*y1**2/Qcoil + w*x2*10**(-9)*y2**2/Qcoil for (x1,y1,x2,y2) in zip(Lp0mm_ys0,Ip,Ls0mm_ys0,Is)]

lpy_df_pred = df_pred.loc[:,'lpy[mm]']
lpx_df_pred = df_pred.loc[:,'lpx[mm]']
wp_df_pred = df_pred.loc[:,'wp[mm]']
a_df_pred = df_pred.loc[:,'a[mm]']
p_df_pred = df_pred.loc[:,'p[mm]']
ls_df_pred = df_pred.loc[:,'ls[mm]']
ws_df_pred = df_pred.loc[:,'ws[mm]']
b = 50 # mm
# Core volume
V_PriCore_temp = []
V_PriCore = []
V_SecCore_temp = []
V_SecCore = []
V_PriCore_ave_temp = []
V_PriCore_ave = []
num_temp = []
num = []
for i in range(len(lpy_df_pred)):
  # Primary
  lpx_i = lpx_df_pred.values[i]
  lpy_i = lpy_df_pred.values[i]
  wp_i = wp_df_pred.values[i]
  a_i = a_df_pred.values[i]
  p_i = p_df_pred.values[i]
  V_PriCore_temp = (lpy_i+2*wp_i+2*a_i)*(lpx_i+2*wp_i+2*a_i)*5/(10**3) # cm3
  V_PriCore.append(V_PriCore_temp) 
  V_PriCore_ave_temp = V_PriCore_temp/(lpy_i+2*wp_i+2*a_i+p_i)*10**3 # cm3/m
  V_PriCore_ave.append(V_PriCore_ave_temp) 
  # Secondary
  ls_i = ls_df_pred.values[i]  
  ws_i = ws_df_pred.values[i]  
  V_SecCore_temp = (ls_i+2*ws_i+2*b)**2*5/(10**3) # cm3
  V_SecCore.append(V_SecCore_temp)   
  # Number of inverter
  num_temp = 1/(lpy_i+2*wp_i+2*a_i+p_i)*10**3 # 1/m
  num.append(num_temp)
  if i % 1000 ==0:
   print(f'Calculating Vcore... {i}/{N}')

# Windings volume
V_PriWind_temp=[]
V_PriWind = []
V_SecWind_temp=[]
V_SecWind = []
V_PriWind_ave_temp = []
V_PriWind_ave = []
wout=[]
for i in range(len(wp_df_pred)):
  # Primary
  lpx_i = lpx_df_pred.values[i]
  lpy_i = lpy_df_pred.values[i]
  wp_i = wp_df_pred.values[i]
  a_i = a_df_pred.values[i]
  p_i = p_df_pred.values[i]
  V_PriWind_temp = (2*(lpx_i+wp_i)+2*(lpy_i+wp_i))*6.6*6.6/(10**3)*Np[i]   
  V_PriWind.append(V_PriWind_temp)
  V_PriWind_ave_temp = V_PriWind_temp/(lpy_i+2*wp_i+2*a_i+p_i)*10**3 # cm3/m
  V_PriWind_ave.append(V_PriWind_ave_temp)
  #Secondary
  ls_i = ls_df_pred.values[i]
  ws_i = ws_df_pred.values[i]
  V_SecWind_temp = 4*(ls_i+ws_i)*6.6*6.6/(10**3)*Ns[i]   
  V_SecWind.append(V_SecWind_temp)
  if i % 1000 ==0:
   print(f'Calculating Vwind... {i}/{N}')
df_pred_processed.insert(0, 'kdiff[%]', kdiff)
df_pred_processed.insert(0, 'V_PriCore_ave[cm3/m]', V_PriCore_ave)
df_pred_processed.insert(0, 'V_PriWind_ave[cm3/m]', V_PriWind_ave)
df_pred_processed.insert(0, 'V_SecCore[cm3]', V_SecCore)
df_pred_processed.insert(0, 'V_SecWind[cm3]', V_SecWind)
df_pred_processed.insert(0, 'Inverter[1/m]', num)
df_pred_processed.insert(0, 'CoilLoss_0mm[W]', CoilLoss_0mm)

df_pred_processed

In [ ]:
df_pred_1 = df_pred_processed
fig_width = 8 #cm # Setting for Conference paper 
fig_height = 3 #cm
font_size = 7 # pt
fig_update = True
marker_size = 5
x_tick_pad = 2
y_tick_pad = 2
x_label_pad = 0.5
y_label_pad = 1

!pip install SciencePlots
import matplotlib
import matplotlib.pyplot as plt
plt.style.reload_library()
plt.style.use(['science','no-latex'])# ref: https://github.com/garrettj403/SciencePlots 
matplotlib.rcParams.update({'font.size': font_size, 'font.family': 'STIXGeneral', 'mathtext.fontset': 'stix'}) # Update the matplotlib configuration parameters:

# distance between x and y axis and the numbers on the axes
matplotlib.rcParams['xtick.major.pad'] = x_tick_pad
matplotlib.rcParams['ytick.major.pad'] = y_tick_pad

def cm2inch(value):
    return value/2.54

def upper_right(xlim, ylim):
    x1 = np.arange(xlim,xlim*10,0.1)
    y1 = ylim
    y2 = ylim*10
    plt.fill_between(x1,y1, y2 ,facecolor='r',alpha=0.3)#alpha is transperancy 
    plt.plot([xlim, xlim], [ylim, ylim*10], 'r--', lw=0.5)    # draw vertical line    
    plt.plot([xlim,xlim*10], [ylim, ylim], 'r--', lw=0.5) # draw horizontal line # Ref: https://www.kite.com/python/docs/matplotlib.pyplot.fill_betwee

def upper_left(xlim, ylim):
    x1 = np.arange(0,xlim+0.1,0.1)
    y1 = ylim
    y2 = ylim*10
    plt.fill_between(x1,y1, y2 ,facecolor='r',alpha=0.3)#alpha is transperancy
    plt.plot([xlim, xlim], [ylim, ylim*10], 'r--', lw=0.5)    # draw vertical line     
    plt.plot([0,xlim], [ylim, ylim], 'r--', lw=0.5)  # draw horizontal line

def lower_right(xlim, ylim):
    x1 = np.arange(xlim,xlim*10,0.1)
    y1 = 0
    y2 = ylim
    plt.fill_between(x1,y1, y2 ,facecolor='r',alpha=0.3)#alpha is transperancy
    plt.plot([xlim, xlim], [0, ylim], 'r--', lw=0.5)    # draw vertical line     
    plt.plot([xlim,xlim*10], [ylim, ylim], 'r--', lw=0.5)  # draw horizontal line

def lower_left(xlim, ylim):
    x1 = np.arange(0,xlim+0.1,0.1)
    y1 = 0
    y2 = ylim
    plt.fill_between(x1,y1, y2 ,facecolor='r',alpha=0.3)#alpha is transperancy
    plt.plot([xlim, xlim], [0, ylim], 'r--', lw=0.5)    # draw vertical line     
    plt.plot([0,xlim], [ylim, ylim], 'r--', lw=0.5)  # draw horizontal line 

In [ ]:
#For Poster
import numpy as np
plt.style.use(['science','no-latex'])
matplotlib.rcParams.update({'font.size': font_size, 'font.family': 'STIXGeneral', 'mathtext.fontset': 'stix'})
x = df_pred_1.loc[:,'Inverter[1/m]']
y = df_pred_1.loc[:,'CoilLoss_0mm[W]']
z = df_pred_1.loc[:,'Bstray[uT(rms)]']

fig1=plt.figure(figsize=(cm2inch(fig_width/2),cm2inch(fig_height)/1.2), dpi=400)
xlim = 1.1 #Number of Inverter [1/m]
ylim = 575 #Coil loss [W] 
lower_left(xlim,ylim)
plt.scatter(x, y, c=z, s=marker_size, cmap ='viridis', vmin = 0, vmax=300,rasterized=True)
plt.colorbar(label=r"$B_{\rm stray}~[\rm \mu T]$")#Show colar bar at the right side 
plt.xlabel(r"Number of inverter [1/m]", labelpad = x_label_pad)              # not shown
plt.ylabel(r'Coil loss [W]', labelpad = y_label_pad) # not shown
plt.axis([0, 1.5, 0, 600])   
plt.show()     
df_pred_2=df_pred_1[(df_pred_1['Inverter[1/m]']<241)&(df_pred_1['CoilLoss_0mm[W]']<600) ]
print(len(df_pred_2))

In [ ]:

plt.style.use(['science','no-latex'])
matplotlib.rcParams.update({'font.size': font_size, 'font.family': 'STIXGeneral', 'mathtext.fontset': 'stix'})
x = df_pred_2.loc[:,'Bstray[uT(rms)]']
y = df_pred_2.loc[:,'kdiff[%]']
z = df_pred_2.loc[:,'V_PriCore_ave[cm3/m]']
fig1=plt.figure(figsize=(cm2inch(fig_width/2),cm2inch(fig_height/1.2)), dpi=400)
xlim = 45 #Bstray[uT]
ylim = 9 #kdiff[%] 
lower_left(xlim,ylim)
plt.scatter(x, y, c=z, s=marker_size, cmap ='viridis', vmin = 0, vmax=6000,rasterized=True)
plt.colorbar(label=r'$V_{\rm PriCore}~[\rm cm^3/m]$')#Show colar bar at the right side 
plt.xlabel(r"$B_{\rm stray}~[\rm \mu T]$",labelpad = x_label_pad) # not shown
plt.ylabel(r"$k_{\rm diff}~[\%]$",labelpad = y_label_pad)              # not shown
plt.axis([0, 200, 0, 20])
plt.show()     
df_pred_3=df_pred_2[(df_pred_2['kdiff[%]']<15)&(df_pred_2['Bstray[uT(rms)]']<45) ]
# print(df_pred_3)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.cbook import get_sample_data
plt.style.use(['science','no-latex'])
matplotlib.rcParams.update({'font.size': font_size, 'font.family': 'STIXGeneral', 'mathtext.fontset': 'stix'})
x = df_pred_3.loc[:,'V_SecCore[cm3]']
y = (df_pred_3.loc[:,'Pave[W/m]'])/1000
z = df_pred_3.loc[:,'V_SecWind[cm3]']
print(x,y,z)
fig1=plt.figure(figsize=(cm2inch(fig_width/2),cm2inch(fig_height/1.2)), dpi=400)
xlim = 1500 #_VSecCore[cm3]
ylim = 30 #Pave[kW] 
upper_left(xlim,ylim)
plt.scatter(x, y, c=z, s=marker_size, cmap ='viridis', vmin = 0, vmax=1000,rasterized=True)
plt.colorbar(label=r'$V_{\rm SecWind}~[\rm cm^3]$')#Show colar bar at the right side 
plt.xlabel(r"$V_{\rm SecCore}~[\rm cm^3]$",labelpad = x_label_pad) # not shown
plt.ylabel(r"$P_{\rm ave}~[\rm kW/m]$",labelpad = y_label_pad)              # not shown
plt.axis([0, 6000, 0, 40]) 
plt.show()
df_pred_4=df_pred_3[(df_pred_3['Pave[W/m]']>30000)&(df_pred_3['V_SecCore[cm3]']<1500)&(df_pred_3['V_SecWind[cm3]']<900) ]
print(len(df_pred_4))

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.cbook import get_sample_data
plt.style.use(['science','no-latex'])
matplotlib.rcParams.update({'font.size': font_size, 'font.family': 'STIXGeneral', 'mathtext.fontset': 'stix'})
x = df_pred_4.loc[:,'VLp[V(rms)]']
y = (df_pred_4.loc[:,'VLs[V(rms)]'])
z = df_pred_4.loc[:,'Ip[A(rms)]']

print(x,y,z)
fig1=plt.figure(figsize=(cm2inch(fig_width/2),cm2inch(fig_height/1.2)), dpi=400)
xlim = 2000 # V(rms)
ylim = 2000 # V(rms) 
lower_left(xlim,ylim)
plt.scatter(x, y, c=z, s=marker_size, cmap ='viridis', vmin = 0, vmax=300,rasterized=True)
plt.colorbar(label=r'$I_{\rm P}~[\rm A(rms)]$')#Show colar bar at the right side 
plt.xlabel(r"$V_{\rm Lp}~[\rm V(rms)]$",labelpad = x_label_pad) # not shown
plt.ylabel(r"$V_{\rm Ls}~[\rm V(rms)]$",labelpad = y_label_pad)              # not shown
plt.axis([0, 3000, 0, 3000]) 
plt.show()
df_pred_5=df_pred_4[(df_pred_4['VLp[V(rms)]']>xlim)&(df_pred_4['VLs[V(rms)]']<ylim)]
print(len(df_pred_5))

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.cbook import get_sample_data
plt.style.use(['science','no-latex'])
matplotlib.rcParams.update({'font.size': font_size, 'font.family': 'STIXGeneral', 'mathtext.fontset': 'stix'})
x = df_pred_5.loc[:,'Ip[A(rms)]']
y = (df_pred_5.loc[:,'Is[A(rms)]'])
z = df_pred_5.loc[:,'VLp[V(rms)]']
fig1=plt.figure(figsize=(cm2inch(fig_width/2),cm2inch(fig_height/1.2)), dpi=400)
xlim = 200 # A(rms)
ylim = 200 # A(rms) 
lower_left(xlim,ylim)
plt.scatter(x, y, c=z, s=marker_size, cmap ='viridis', vmin = 0, vmax=2000,rasterized=True)
plt.colorbar(label=r'$V_{\rm LP}~[\rm A(rms)]$')#Show colar bar at the right side 
plt.xlabel(r"$I_{\rm P}~[\rm A(rms)]$",labelpad = x_label_pad) # not shown
plt.ylabel(r"$I_{\rm S}~[\rm A(rms)]$",labelpad = y_label_pad)              # not shown
plt.axis([0, 300, 0, 300]) 
plt.show()
df_pred_6=df_pred_5[(df_pred_5['Ip[A(rms)]']>xlim)&(df_pred_5['Is[A(rms)]']<ylim)]
print(len(df_pred_6))
df_pred_6